### Imports and device for Assignment

In [ ]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader, Subset
import numpy as np
from collections import Counter
import  torch.nn as nn
from torchsummary import summary
from  torchvision.models import mobilenet_v2, MobileNet_V2_Weights

In [365]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("The device is:", device)

The device is: cpu


### Task 1: Prepare Data Subset
- Use the CIFAR-10 training set and construct a balanced subset with 1000 images per class (total = 10,000 images).
- To download the dataset, you may use following two packages in Pytorch: import torch, torchvision in the beginning of the code, and you can refer to following code to download the training set and testing set of the dataset:
    - full_trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    - testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
- After download and load the dataset, you will need to write you own code to make sure each class has 1000 images randomly selected from the whole training set for training. Testing set remain the same as the original dataset test set.
- Use a fixed random seed to ensure reproducibility.
- Verify that each class has the correct number of samples.

In [ ]:
# 1. Set random seed
seed = 7
np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
# 2. Transformation of CIFAR 10 dataset
transform = transforms.Compose([
    transforms.ToTensor()
])

# 3. Downloading the dataset
full_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform )

100%|██████████| 170M/170M [02:59<00:00, 950kB/s]  


In [ ]:
# 4. Select 1000 samples per class

# Dictionary to hold indices per class
# CIFAR 10 has 10 unique classes
class_indices = {i: [] for i in range(10)}  

# Iterate through dataset and collect indices for each class
for idx, (_, label) in enumerate(full_trainset):
    if len(class_indices[label]) < 1000:
        class_indices[label].append(idx)

    # Stop iteration if all classes collected 1000 samples
    if all(len(v) == 1000 for v in class_indices.values()):
        break

# Flatten the list of indices to create a subset
selected_indices = [idx for indices in class_indices.values() for idx in indices]

# Create a subset dataset
subset_trainset = Subset(full_trainset, selected_indices)


In [281]:
# 5. Verify class distribution
subset_targets = [full_trainset.targets[i] for i in selected_indices]
print("Class distribution in subset:", Counter(subset_targets))
print("Total subset size:", len(subset_trainset))

Class distribution in subset: Counter({0: 1000, 1: 1000, 2: 1000, 3: 1000, 4: 1000, 5: 1000, 6: 1000, 7: 1000, 8: 1000, 9: 1000})
Total subset size: 10000


In [305]:
# 6. Setting data and target variables for further use in training and testing model
# Training dataset
trainset_target = np.array([full_trainset.targets[i] for i in selected_indices])
trainset_data = np.array([full_trainset.data[i] for i in selected_indices])

# Test dataset
testset_target = testset.targets
testset_data = testset.data

### Task 2: Implement a Custom CNN
- Design a custom CNN model using at least 3 convolutional layers.
- Use ReLU activation, pooling, and optionally batch normalization or dropout.
- Structure your model cleanly and modularly.

In [414]:
def get_model():
    model = nn.Sequential(
        # First convolution layer
        nn.Conv2d(3, 32, kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d(2),

        # Second convolution layer
        nn.Conv2d(32, 64, kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d(2),

        # Third convolution layer
        nn.Conv2d(64, 128, kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d(2),

        # Fully connected layer
        nn.Flatten(),

        # Output layer
        nn.Linear(512, 10)
    ).to(device)

    return model

In [447]:
model = get_model()
summary(model, (3, 32,32));

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 32, 30, 30]          896
├─ReLU: 1-2                              [-1, 32, 30, 30]          --
├─MaxPool2d: 1-3                         [-1, 32, 15, 15]          --
├─Conv2d: 1-4                            [-1, 64, 13, 13]          18,496
├─ReLU: 1-5                              [-1, 64, 13, 13]          --
├─MaxPool2d: 1-6                         [-1, 64, 6, 6]            --
├─Conv2d: 1-7                            [-1, 128, 4, 4]           73,856
├─ReLU: 1-8                              [-1, 128, 4, 4]           --
├─MaxPool2d: 1-9                         [-1, 128, 2, 2]           --
├─Flatten: 1-10                          [-1, 512]                 --
├─Linear: 1-11                           [-1, 10]                  5,130
Total params: 98,378
Trainable params: 98,378
Non-trainable params: 0
Total mult-adds (M): 5.08
Input size (MB): 0.01
Forward/backward pass siz

### Task 3: Load and Adapt MobileNetV2
- Load a pretrained MobileNetV2 from torchvision.models.
- Modify the classifier to output 10 classes (CIFAR-10).
- Properly initialize the classifier layer and allow appropriate fine-tuning.

In [469]:
MobileNetV2_model = mobilenet_v2(weights=MobileNet_V2_Weights.IMAGENET1K_V1)
# Since CIFAR 10 has 10 classes, we need to modify the classifier output to 10

'''
# Method - 1
# 1. Get the input features of the classifier
# 2. then replace with a new Linear layer for 10 classes
'''
in_features = MobileNetV2_model.classifier[1].in_features 
MobileNetV2_model.classifier[1] = nn.Linear(in_features, 10) 

# Method - 2
# MobileNetV2_model.classifier[1].out_features = 10


In [470]:
# Fine-tuning the MobileNetV2 model
for param in model.parameters():
    param.requires_grad = True


In [289]:
class CIFAR10Dataset(Dataset):
    def __init__(self, images, targets):
        images = torch.tensor(images)
        images = images.float()/255
        self.images = images
        self.targets = targets
    
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.targets[idx]
        return image, label

In [286]:
def get_training_data():
    train = CIFAR10Dataset(trainset_data, trainset_target)
    train_dl = DataLoader(train, batch_size=1000, shuffle=True)
    return train_dl

In [287]:
def get_test_data():
    test = CIFAR10Dataset(testset_data, testset_target)
    test_dl = DataLoader(test, batch_size=1000, shuffle=True)
    return test_dl